In [1]:
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from statistics import mean

import torch as th

from tabpfn import TabPFNClassifier

from tab_pfn.networks import SCM, TabPFN
from tab_pfn.metrics import AccuracyMeter

import warnings

warnings.filterwarnings("ignore")

In [25]:
classifier = TabPFNClassifier(device='cuda', N_ensemble_configurations=32)

n_datasets = 1024

accuracy_meter = AccuracyMeter(None)
nb = 0

for _ in tqdm(range(n_datasets)):
    scm = SCM(100, (2, 10), False)
    x, y = scm(1024)
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)
    
    try:
        classifier.fit(x_train, y_train)
        p_eval = th.tensor(classifier.predict_proba(x_test))[None]
        
        accuracy_meter.add(p_eval.cpu(), y_test[None])
    except ValueError:
        pass

  0%|          | 0/1024 [00:00<?, ?it/s]/home/samuel/PycharmProjects/TabPFN/venv/lib/python3.11/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
  0%|          | 1/1024 [00:01<25:30,  1.50s/it]/home/samuel/PycharmProjects/TabPFN/venv/lib/python3.11/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on t

In [26]:
print(accuracy_meter.accuracy())

0.7545524523601065


In [23]:
my_tab_pfn = TabPFN(100, 10, 256, 512, 1024, 4, 12)
my_tab_pfn.load_state_dict(th.load("/home/samuel/PycharmProjects/TabPFN/out/model_52223.pt"))
my_tab_pfn.cuda()

n_datasets = 1024

accuracy_meter = AccuracyMeter(None)
nb = 0

for _ in tqdm(range(n_datasets)):
    scm = SCM(100, (2, 10), True)
    scm.cuda()
    x, y = scm(1024)
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)
    x_train= x_train[None]
    y_train= y_train[None]
    x_test= x_test[None]
    y_test= scm.inverse_class_permutations()[y_test][None]
    
    try:
        p_eval = my_tab_pfn(x_train, y_train, x_test)[:, :, scm.get_class_index()]
        
        accuracy_meter.add(p_eval.cpu(), y_test.cpu())
    except ValueError:
        pass

100%|██████████| 1024/1024 [00:31<00:00, 32.69it/s]


In [24]:
accuracy_meter.accuracy()

0.6238009661612426